# Expérimentation V1 : Modèle Hybride CatBoost

Ce notebook implémente la stratégie **Supervisé Multiclass + Smart Masking** définie dans `notes/nouveau_model.md`.

### Objectifs
1. Préparer un dataset d'entraînement via **Smart Masking** (sur-échantillonnage des produits rares).
2. Entraîner un **CatBoostClassifier** qui utilise nativement les features catégorielles.
3. Évaluer la performance brute vs **Hybride** (combinaison avec la Baseline).


In [ ]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Ajout du dossier src au path
project_root = Path("..").resolve()
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

from src.config import DATA_DIR, ARTIFACTS_DIR
from src.models.catboost.dataset import CatboostDatasetBuilder
from src.models.catboost.trainer import CatboostTrainer
from src.models.catboost.predictor import HybridPredictor
from src.pipelines.baseline_pipeline import BaselineArtifact
from src.evaluation.evaluate_V2 import evaluate_masking

## 1. Chargement et Nettoyage

In [ ]:
train_path = DATA_DIR / "Train.csv"
test_path = DATA_DIR / "Test.csv"

df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)

# Définition des colonnes (même logique que Baseline)
PROFILE_COLS = [
    "ID","join_date","sex","marital_status","birth_year",
    "branch_code","occupation_code","occupation_category_code"
]
PRODUCT_COLS = [c for c in df_train.columns if c not in PROFILE_COLS]

print(f"Produits ({len(PRODUCT_COLS)}): {PRODUCT_COLS}")

In [ ]:
def normalize_cat(s: pd.Series) -> pd.Series:
    return s.astype(str).str.strip().str.casefold()

def apply_cleaning(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    # Normalisation des catégorielles
    for c in ["sex", "marital_status", "branch_code", "occupation_code", "occupation_category_code"]:
        out[c] = normalize_cat(out[c])
    
    # Join Year
    jy = pd.to_numeric(out["join_date"].astype(str).str.split("/").str[-1], errors="coerce")
    out["join_year"] = jy.fillna(2017) # Fillna moyenne/mode rapide
    
    # Age
    out["age"] = (out["join_year"] - out["birth_year"]).clip(18, 90).fillna(38)
    
    return out

df_train_clean = apply_cleaning(df_train)
df_test_clean = apply_cleaning(df_test)

## 2. Préparation du Dataset Supervisé

Nous divisons le train en deux :
- **Train Split** (80%) : Pour entraîner CatBoost avec Smart Masking.
- **Val Split** (20%) : Pour évaluer la performance Hybride (produit masqué).

In [ ]:
from sklearn.model_selection import train_test_split

# On garde uniquement les clients avec au moins 2 produits pour le 'val' (pour pouvoir en cacher 1)
# Mais pour le train, on peut utiliser tout le monde (même taille 1 pour apprendre le profil? Non, besoin de target)
# Le dataset builder filtre automatiquent min_basket=2

ids_unique = df_train_clean["ID"].unique()
train_ids, val_ids = train_test_split(ids_unique, test_size=0.2, random_state=42)

df_local_train = df_train_clean[df_train_clean["ID"].isin(train_ids)].copy()
df_local_val = df_train_clean[df_train_clean["ID"].isin(val_ids)].copy()

print(f"Train local: {df_local_train.shape}, Val local: {df_local_val.shape}")

In [ ]:
# Configuration du Builder
CAT_FEATURES = ["sex", "marital_status", "branch_code", "occupation_code", "occupation_category_code"]

builder = CatboostDatasetBuilder(
    product_cols=PRODUCT_COLS,
    cat_cols=CAT_FEATURES,
    random_state=42
)

print("Construction du dataset train avec Smart Masking...")
X_train, y_train = builder.build_dataset(df_local_train, strategy='smart', min_basket=2)

print(f"X_train shape: {X_train.shape}")
print(X_train.head(3))

## 3. Entraînement CatBoost

In [ ]:
# Correction : passage des arguments directement sans dictionnaire 'params'
trainer = CatboostTrainer(
    cat_features=CAT_FEATURES,
    iterations=500,
    learning_rate=0.1,
    depth=6
)

trainer.fit(X_train, y_train)

## 4. Évaluation Hybride (Validation)

On charge la baseline pour l'hybridation.

In [ ]:
# Chargement Baseline Artifact
baseline_art = BaselineArtifact.load(ARTIFACTS_DIR / "baseline_v0")

# Création du Prédicteur Hybride
hybrid_model = HybridPredictor(trainer, baseline_art)

In [ ]:
# Préparation des données de validation pour evaluate_masking
# evaluate_masking attend X_full (produits) et utilise score_fn(x, idx)
# Pour que score_fn accède au contexte, on a besoin d'accéder à df_local_val via idx

# On doit s'assurer que les indices correspondent.
df_local_val = df_local_val.reset_index(drop=True)
X_val_full = df_local_val[PRODUCT_COLS].values

# Fonction de scoring wrapper
def hybrid_scorer(x_row_obs, idx_original):
    # 1. Reconstruire le contexte (profil)
    context_row = df_local_val.iloc[[idx_original]].copy()
    
    # 2. Mettre à jour les produits observés dans le contexte
    # (car evaluate_masking a masqué un produit dans x_row_obs)
    context_row[PRODUCT_COLS] = x_row_obs # Broadcast
    
    # 3. Prédiction (alpha=0.3 par exemple)
    # predict_proba attend un DataFrame
    probas = hybrid_model.predict_proba(context_row, alpha=0.3)
    return probas[0]

# Évaluation
print("Évaluation sur le set de validation...")
info, metrics = evaluate_masking(
    X_full=X_val_full, 
    score_fn=hybrid_scorer, 
    hide_k=1, 
    min_observed=1
)

print("\n--- Résultats Hybride (Alpha=0.3) ---")
print(metrics)

## 5. Analyse de l'Importance des Features

In [ ]:
fi = trainer.model.get_feature_importance(prettified=True)
print(fi.head(10))

plt.figure(figsize=(10, 6))
pl = fi.head(15)
plt.barh(pl["Feature Id"], pl["Importances"])
plt.gca().invert_yaxis()
plt.title("Top 15 Feature Importance")
plt.show()

## 6. Comparaison Finale : Baseline vs Hybride

C'est le moment de vérité. On compare la Baseline pure (V0) contre notre nouveau modèle Hybride (V1) sur le même set de validation.
On s'attend à ce que l'Hybride soit meilleur, surtout sur le Hit@1.

In [ ]:
# 1. Évaluer la Baseline Pure sur le MÊME set de validation
print("Évaluation de la Baseline Pure...")
def baseline_scorer(x_row_obs, idx_original):
    # Baseline ignore idx_original et contexte
    return baseline_art.cond.score_one(x_row_obs)

_, metrics_bl = evaluate_masking(
    X_full=X_val_full, 
    score_fn=baseline_scorer, 
    hide_k=1, 
    min_observed=1
)

# 2. Comparaison
comp_df = pd.DataFrame({
    "Baseline (V0)": metrics_bl,
    "Hybride (V1)": metrics
})
comp_df["Gain"] = comp_df["Hybride (V1)"] - comp_df["Baseline (V0)"]

print("\n=== Comparaison des Performances ===")
print(comp_df.round(4))

# Petit plot
comp_df[[ "Baseline (V0)", "Hybride (V1)"]].plot(kind='bar', figsize=(10, 5))
plt.title("Performance par Métrique : V0 vs V1")
plt.ylabel("Score")
plt.grid(axis='y', alpha=0.3)
plt.show()